# Method
- Simple method of using regular seasons win-loss ratio between pairs of team as probablity of winning.
- Example if Team A and Team B has 5 matches in total and Team A wins 3 out of 5, the probability of Team A winning is 0.6
- Probability of winning is 0.5 if A and B has no historical match up.

In [ ]:
import pandas as pd

In [ ]:
DIR = '../input/mens-march-mania-2022/MDataFiles_Stage1'
df_teams = pd.read_csv(f'{DIR}/MTeams.csv')
df_seasons = pd.read_csv(f'{DIR}/MSeasons.csv')
df_tseeds = pd.read_csv(f'{DIR}/MNCAATourneySeeds.csv')
df_tcompresults = pd.read_csv(f'{DIR}/MNCAATourneyCompactResults.csv')
df_rcompresults = pd.read_csv(f'{DIR}/MRegularSeasonCompactResults.csv')

In [ ]:
df_rcompresults['Team1'] = df_rcompresults.apply(lambda x: x[['WTeamID', 'LTeamID']].min(), axis = 1)
df_rcompresults['Team2'] = df_rcompresults.apply(lambda x: x[['WTeamID', 'LTeamID']].max(), axis = 1)
df_rcompresults['Team1Win'] = df_rcompresults['WTeamID'] == df_rcompresults['Team1']

In [ ]:
df_prep_sub = df_rcompresults.groupby(['Team1', 'Team2'], as_index = False).agg({'Team1Win':['sum','count']})
df_prep_sub.columns = ['Team1', 'Team2', 'Team1Win', 'TotalMatches']
df_prep_sub['Team1'] = df_prep_sub['Team1'].astype('string')
df_prep_sub['Team2'] = df_prep_sub['Team2'].astype('string')
df_prep_sub['Teams'] = df_prep_sub[['Team1', 'Team2']].agg('_'.join, axis=1)
df_prep_sub['Pred'] = df_prep_sub['Team1Win']/df_prep_sub['TotalMatches']

In [ ]:
df_sub = pd.read_csv(f'{DIR}/MSampleSubmissionStage1.csv')
df_sub['Teams'] = df_sub['ID'].apply(lambda x: '_'.join(x.split('_')[1:]))
df_sub = pd.merge(df_sub.drop('Pred', axis = 1), df_prep_sub, on = 'Teams', how = 'left')
df_sub = df_sub.loc[:, ['ID', 'Pred']].fillna(0.5)
df_sub.to_csv('submission.csv', index = False)